# 유튜브 랭킹 데이터 수집해서 시각화하기

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [5]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
#time.sleep(2)

In [10]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [11]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [12]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [23]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
category


'[음악/댄스/가수]'

In [16]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [18]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]

subscriber, view, video

('6400만', '190억0381만', '371')

In [37]:
channels = []

for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip()
  name = channel.select_one('.subject a').text.strip()
  subscriber = channel.select_one('.subscriber_cnt').text
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.video_cnt').text

  channels.append([category,name,subscriber,view,video])


In [38]:
df = pd.DataFrame(channels, columns=['카테고리', '채널명', '구독자수', '뷰수', '비디오수'])
df.head()

,카테고리,채널명,구독자수,뷰수,비디오수
0,[음악/댄스/가수],BLACKPINK,6400만,190억0381만,371개
1,[음악/댄스/가수],HYBE LABELS,6030만,187억1305만,654개
2,[음악/댄스/가수],BANGTANTV,5640만,121억9496만,"1,579개"
3,[음악/댄스/가수],SMTOWN,2850만,218억7092만,"3,729개"
4,[키즈/어린이],Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개


In [39]:
df.tail()

,카테고리,채널명,구독자수,뷰수,비디오수
95,[음식/요리/레시피],까니짱 [ G-NI ],408만,10억3471만,482개
96,[키즈/어린이],로미유 스토리[Romiyu Story],408만,10억6557만,433개
97,[음악/댄스/가수],NCT DREAM,386만,3억7422만,228개
98,[BJ/인물/연예인],허팝Heopop,381만,31억3892만,"1,847개"
99,[애완/반려동물],크림히어로즈,381만,13억7401만,939개


- 만과 억을 숫자로 바꿔주는 함수

In [35]:
def convert_unit(s):
  #s = ''.join(s.split('억'))
  s = s.replace('억', '').replace('개','').replace(',', '')
  s = s.replace('만', '0000')
  return f'{int(s):,d}'
    

In [36]:
convert_unit('190억0381만')

'19,003,810,000'

- 2번째 페이지 -XPath로 찾아 Click() 하여 이동하기

In [40]:

driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
# //*[@id="list-skin"]/nav/span/a[2]


In [41]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [44]:
channel_list = soup.select('.aos-init')

In [45]:
results = []

for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip()
  name = channel.select_one('.subject a').text.strip()
  subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
  view = convert_unit(channel.select_one('.view_cnt').text)
  video = convert_unit(channel.select_one('.video_cnt').text)

  results.append([category,name,subscriber,view,video])


In [46]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '뷰수', '비디오수'])
df.head()

,카테고리,채널명,구독자수,뷰수,비디오수
0,[취미/라이프],JaeYeol ASMR 재열,"3,800,000","1,011,540,000",970
1,[BJ/인물/연예인],워크맨-Workman,"3,780,000","679,370,000",141
2,[BJ/인물/연예인],waveya 2011,"3,730,000","1,346,280,000",702
3,[키즈/어린이],Lime Tube[라임튜브],"3,730,000","2,595,420,000","1,723"
4,[음악/댄스/가수],WINNER,"3,730,000","1,171,260,000",428


# page1~10 까지 크롤링

In [47]:
results = []

for page in range(1,11):
  url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
  driver.get(url)
  time.sleep(2)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  channel_list = soup.select('.aos-init')

  for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)

    results.append([category,name,subscriber,view,video])


In [51]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '뷰수', '비디오수'])
df

,카테고리,채널명,구독자수,뷰수,비디오수
0,[음악/댄스/가수],BLACKPINK,"64,000,000","19,003,810,000",371
1,[음악/댄스/가수],HYBE LABELS,"60,300,000","18,713,050,000",654
2,[음악/댄스/가수],BANGTANTV,"56,400,000","12,194,960,000","1,579"
3,[음악/댄스/가수],SMTOWN,"28,500,000","21,870,920,000","3,729"
4,[키즈/어린이],Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...
995,[게임],미소,"510,000","210,770,000","3,084"
996,[미분류],밉지않은 관종언니,"510,000","67,740,000",170
997,[미분류],복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,[음식/요리/레시피],Muggo,"510,000","183,720,000","1,388"


In [53]:
df.to_csv('유튜브랭크1000.csv', index=None) # index=False 도 됨

In [54]:
driver.close()